---
---

<center><h1> Setting up the Environment.</h1></center>

---


#### `Generate Transaction Data`

---

In this notebook, we will generate random transaction data for the `transaction` table.

---

In the `transaction` table, we have the following columns:

- **`transaction_id`**    : Randomly generated string for each of the transaction. <br>
- **`user_id`** : Random user id from the existing users. A list of user id will be updated every minute. <br>
- **`product_id`**  : Random product id from the CSV file <br>
- **`transaction_time`**     : Current time will be used as transaction time <br> 
- **`price`**   : price of the product will be according to the product_id using the CSV file <br>

---



---

Every 4 seconds a row in added into the transaction table.

----

#### `Importing the Required Libraries`

---

In [7]:
# importing the required libraries
import time
import random
import uuid
import pandas as pd
import mysql.connector as mysql

---

Read the `product_table.csv` which is present in the dataset folder. We will use this table to add `product_id` and `price` for every transaction.


---

In [8]:
# read the products CSV file
products_table = pd.read_csv('dataset/product_table.csv')

---

####  `Define a function to Generate New Transactions.`

---

In [9]:
# function to generate new transactions
# this function will use a list of existing users as a parameter
def generate_new_transaction(users):
    
    # generate random transaction id
    transaction_id = "TI-"+str(uuid.uuid4())
    
    # select a random user id 
    user_id = users[random.randint(0,len(users))][0]
    
    # select a random product_id using the CSV file
    random_product = random.randint(0, products_table.shape[0]-1)
    product_id = products_table.iloc[random_product]['product_id']
    
    # current time will be transaction time
    transaction_time = str(pd.datetime.now())
    
    # price of the product using the CSV file
    price = products_table.iloc[random_product]['product_price']
    
    # return the values
    return transaction_id, user_id, product_id, transaction_time, price
    

#### `Connect to the database`

---

In [10]:
# connect to the database. 
# enter the details according to your mysql configuration
db = mysql.connect(
    host = "localhost",
    user = "lakshay",  ## Enter your user name here
    password = "ABC@123", ## Enter your password here
    database = "website", ## Enter the database name here
    auth_plugin = "mysql_native_password",
)
# create the cursor
cursor = db.cursor()

---

Now, The new users data is continously coming with our first script. Here, we will first extract the `user_id` from the user table. We will then do 25 transaction, each transaction will happen after every 4 seconds. And after the 25 transactions, we will again update the users table.

---

In [ ]:
while True:
    # get the users list
    command = "SELECT user_id FROM users"
    cursor.execute(command)
    users = cursor.fetchall() 
    # create the new random transaction
    # a time is selected randomly upto 4 seconds and transction row will be added.
    # after 25 transactions, again all the users list will be updated.
    for i in range(25):
        command = "INSERT INTO transaction values" + str(generate_new_transaction(users))
        cursor.execute(command)
        db.commit()
        time.sleep(random.randint(0,4))